## Objetivo
1. Fazer scrapy de uma pagina de noticias
2. Deixa somente os conteudo relevantes
3. Armazenar os dados no BigQuery

## 0. Importando as bibliotecas

In [20]:
import pandas_gbq
from functions import *
from gcp_auth import credentials

## 1. Configurando o Webscraping

1. Primeiro, o código define duas variáveis:
    * `url`: Armazena a URL da página que será acessada, neste caso, a página da Globo.
    * `headers`: É um dicionário que contém informações sobre o navegador que está fazendo a requisição. Aqui, define o `User-Agent` como "Mozilla/5.0", simulando um navegador comum.

1. Obtendo o HTML da página:
    * A linha `response = requests.get(url, headers=headers)` usa a biblioteca requests para fazer uma requisição GET à URL especificada no url.
1. Extraindo dados com BeautifulSoup:

    * A linha `page = BeautifulSoup(response.text, 'html.parser')` usa a biblioteca BeautifulSoup para transformar o texto HTML da resposta em um objeto que pode ser facilmente navegado e analisado.
1. Selecionando a tag dos links:
    * Por fim, o código usa o método `find` do objeto `soup` para encontrar a todas as tag (`a`) na página.

In [21]:
url = "https://www.globo.com/"
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
page = BeautifulSoup(response.text, "html.parser")
    
urls = page.find_all("a")

5. Selecionando somente as tags que contem links de noticias 
* As variaveis `tag_class1` e `tag_class2` armazenam strings que representam nomes de classes em elementos HTML.
*  Essas classes são usadas para identificar elementos que contêm links para artigos de notícias utilizadas pela Globo.

In [22]:
tag_class1 = "post__title"
tag_class2 = "post-multicontent__link--title__text"

news_url_list = []
for url in urls:
    if(url.h2 != None) and (url.h2.get("class") != None):
        if tag_class1 in url.h2.get("class"):
            news_url_list.append(url.get("href"))
        elif tag_class2 in url.h2.get("class"):
            news_url_list.append(url.get("href"))

print(f'Quantidade de notícias: {len(news_url_list)}')

Quantidade de notícias: 64


## 2. Deduplicacao dos dados

Salvei o dataset que estava no BigQuery em uma planilha, em uma variavel chamada `df_scrapy_news_antigo`

In [23]:
df_scrapy_news_antigo = pd.read_csv("data/bigquery_news_data.csv") 

1. Transformo a variável `news_url_list` em um DataFrame chamado `df_urls`.
1. Comparo o DataFrame `df_urls` com o DataFrame `df_scrapy_news_antigo` com base na coluna <strong>url</strong>, retornando assim somente as URLs novas, que não estão no BigQuery.


In [24]:
df_urls = pd.DataFrame(news_url_list, columns=['url']) 
df_url_deduplicado = df_urls[~df_urls['url'].isin(df_scrapy_news_antigo['url'])]

Transformo o dataframe `df_url_deduplicado` em uma lista para conseguir usar a funcao `df_news_data`

In [25]:
list_urls_novos = df_url_deduplicado['url'].tolist()
print(f'Quantidade de notícias novas: {len(list_urls_novos)}')

Quantidade de notícias novas: 0


## 3. Raspagem do conteúdo das notícias

Nessa parte estou utilizando uma função chamada `df_news_data`, onde os processos estão mais detalhados abaixo

Processos:

1. Loop: A função df_news_data utiliza um loop para iterar por cada URL na lista <strong>list_urls_novos.</strong>

1. Requisição HTTP: Para cada URL, a função faz uma requisição HTTP para recuperar o conteúdo HTML da página de notícia.

1. Análise de HTML: O HTML da página de notícia é então analisado para extrair os dados desejados:

    * url: A URL da própria notícia.
    * date: A data da publicação da notícia.
    * author: O autor da notícia (se disponível).
    * title: O título da notícia.
    * subtitle: O subtítulo da notícia (se disponível).
    * text: O corpo principal do texto da notícia.
    
1. Armazenamento de Dados: Os dados extraídos para cada notícia são armazenados em um dicionário.

1. Agregação de Dados: O dicionário é transformado em um DataFrame.

In [26]:
df_scrapy_news_novo = df_news_data(list_urls_novos)
print(f'Quantidade de notícias novas coletas: {len(df_scrapy_news_novo)}')

Inicio da raspagem de dados
Coleta finalizada
Quantidade de notícias novas coletas: 0


## 4. Salvo os dados novos

Concateno o dataset ja existente `df_scrapy_news_antigo` com o dataset com as novas URLs `df_scrapy_news_novo` e salvo novamente na mesma planilha

In [27]:
# Concatenar os DataFrames df_scrapy_news_antigo e df_new__novo_deduplicado 
df_news_concat = pd.concat([df_scrapy_news_antigo, df_scrapy_news_novo]) 
df_news_concat.to_csv("data/bigquery_news_data.csv", index=False, sep=',')

## 5. Adiciono os novos dados coletados no BigQuery

In [28]:
# Insere os dados novos no BigQuery
print(f'Inserindo {len(df_scrapy_news_novo)} notícias novas no BigQuery')
pandas_gbq.to_gbq(df_scrapy_news_novo, 
                  "bigquery_news.tb_news_data", 
                  project_id="project-scrapy-news",
                  if_exists="append",
                  credentials=credentials)

Inserindo 0 notícias novas no BigQuery
